In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import floor
import math
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os.path

# TODO CELL VELOCITIES
# TODO INSPECT THE MONOMERS; WRITE MONOMER CODE

# Setup

In [ ]:
with_obstacles = False



data_from = ''
path = "Data/" + data_from

csv = ".csv"

def getConstants(config):
    name = 'constants.csv'
    constants = pd.read_csv("Data/" + config + name)
    out = {}
    out['avg'] = int(constants['average_particles_per_cell'])
    #out['t'] = int(constants['timesteps'])
    out['delta_t'] = float(constants['time_lapse'])
    out['a'] = int(constants['cell_dim'])
    out['w'] = int(constants['width'])
    out['h'] = int(constants['height'])
    out['num_particles'] = int(constants['total_number_of_particles'])
    out['m'] = int(constants['particle_mass'])
    out['k_BT'] = float(constants['k_BT'])
    out['g'] = float(constants['g'])
    return out

constants = getConstants(data_from)
x_dim = constants['w']
y_dim = constants['h']

# Load Stuff

In [ ]:
def load_monomers(path, constants):
    monomers_t = []

    # Loading particles
    print('Loading Monomers ..')
    filenames_monomers = glob.glob('{}*.csv'.format(path))
    for it, file in enumerate(filenames_monomers):
        monomers_t.append(pd.read_csv(file))
        if (it % 500 == 0):
            print('--loaded {}'.format(it))
    print('Monomers loaded and saved!\n')
    # Monomers loaded
    return monomers_t


def load_particles(path):
    num_timesteps = len(monomers_t)

    # Loading particles
    print('Loading particles ..')
    filenames_particles = glob.glob('{}*.csv'.format(path))
    particles_t = []
    for file in filenames_particles:
        num = int(file.split('timestep')[1].replace(".csv", ""))
        if (num > num_timesteps - stationary):
            particles_t.append(pd.read_csv(file))
            if (num % 500 == 0):
                print('--loaded {}'.format(num))
    print('Particles loaded!\n')# and saved
    # Particles loaded
    
    return particles_t



def load_cells(path, columns, shown_rows, shown_cols):
    print('Loading cells')
    cells_timesteps = []

    filenames_cells = glob.glob('{}*.csv'.format(path))
    for it, file in enumerate(filenames_cells):
        if it > num_timesteps - stationary:
            df = pd.read_csv(file)
            cells_timesteps.append(df)
            if (it % 500 == 0):
                print('--loaded {}'.format(it))
        it += 1
    print('Cells loaded and saved!\n')
    # Cells loaded
    return cells_timesteps
    

monomers_t = load_monomers(path + f'monomers_av{constants["avg"]}', constants)
particles_t = load_particles(f'Data/particles_av{constants["avg"]}')
cells_path = path + 'cells_av{}'.format(constants['avg'])
cells_timesteps = load_cells(cells_path, columns, shown_rows, shown_cols)
I,J,U,V,pivots = prepare_cells(cells_timesteps, columns, shown_rows, shown_cols)

In [ ]:
num_timesteps = len(monomers_t) - 2
stationary = 1000 # last 1000

In [ ]:
x_region = (100, 200)
y_region = (0, 20)

def process_plotdata(x_region, y_region):
    x0_obstacles = x_region[0]
    x1_obstacles = x_region[1]
    y0_obstacles = y_region[0]
    y1_obstacles = y_region[1]
    x = np.arange(x0_obstacles, x1_obstacles)
    y = np.arange(y0_obstacles, y1_obstacles)
    X, Y = np.meshgrid(x, y)
    cells = cells_timesteps[-1]
    pivot = cells.reset_index().pivot(index = 'i', columns = 'j', values = 'num')
    vx = np.array([[cells['meanX'].loc[row, col] for col in x] for row in y])
    vy = np.array([[cells['meanY'].loc[row, col] for col in x] for row in y])
    return X, Y, vx, vy, pivot

X, Y, vx, vy, pivot = process_plotdata(x_region, y_region)

In [ ]:
print('Plotting data ..')
fig = plt.figure(figsize=(20,20))
ax = [fig.add_subplot(2,2,i+1) for i in range(4)]


for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    #a.set_aspect('equal')
    
fig.subplots_adjust(wspace=0, hspace=0)

color = np.sqrt(vx**2 + vy**2)
point_size = 0.3

if with_obstacles:
    for index, o in obstacles.iterrows():
        circle1 = plt.Circle((o['x'], o['y']), obstacle_radius, color = 'orange')
        circle2 = plt.Circle((o['x'], o['y']), obstacle_radius, color = 'orange')
        #circle3 = plt.Circle((o['x'], o['y']), obstacle_radius, color = 'black')
        #circles.append(circle)
        ax[1].add_artist(circle1)
        ax[2].add_artist(circle2)
        #ax[3].add_artist(circle3)
        #ax[0].xaxis.set_ticks([])

#ax[0].xaxis.set_ticks([])
#ax[0].yaxis.set_ticks([])
ax[0].quiver(X, Y, vx, vy, color)
ax[0].set(xlim=x_region, ylim=y_region)

#ax[1].xaxis.set_ticks([])
#ax[1].yaxis.set_ticks([])
ax[1].streamplot(X, Y, vx, vy, color=color, density=streamplot_density) # grid
ax[1].set(xlim=x_region, ylim=y_region)

#ax[2].xaxis.set_ticks([])
#ax[2].yaxis.set_ticks([])
particles = particles_t[-1]
ax[2].plot(particles['x'], particles['y'], "o", markersize = point_size)
monomers_T = monomers_t[-1]
ax[2].plot(monomers_T['x'], monomers_T['y'], 'o', markersize = point_size * 20, color = 'red')
ax[2].set(xlim=x_region, ylim=y_region)

#ax[3].xaxis.set_ticks([])
#ax[3].yaxis.set_ticks([])
#img = ax[3].imshow(pivot, cmap='hot')
#fig.colorbar(img, ax=ax[3], fraction=0.046, pad=0.005)
sns.heatmap(pivot, ax=ax[3], xticklabels = False, yticklabels = False, cbar_kws={"fraction": 0.046, "pad": 0.005})
ax[3].set(xlim=x_region, ylim=y_region)
#ax[3].xticks('')
#ax[3].yticks('')
ax[3].set_ylabel('')
ax[3].set_xlabel('')
#ax[1,1].imshow(pivot, cmap='hot')

plt.savefig("Assets/stationary_region.png")
#plt.close()
print('Data plotted and saved!')

In [ ]:
def viscosity(v_max, m = 1, av_N_c = 10, g = 0.01, D = 50):
    return (m * av_N_c * g * D**2) / (8 * v_max)

def parabolic_flow(y, D, v_max = 0.1):
    return (4 * v_max * (D - y) * y)/(D**2)

def getLastCell(data_from):
    path = "Data/" + data_from
    constants = getConstants(data_from)
    try:
        cells = pd.read_csv(path + f"cells_av{constants['avg']}_timestep{num_timesteps}.csv")
    except FileNotFoundError as f:
        cells = pd.read_csv(path + f"cells_av{constants['avg']}_timestep0{num_timesteps}.csv")
    return cells

def getLastCells(data_from, num):
    path = "Data/" + data_from
    constants = getConstants(data_from)
    start_timestep = num_timesteps - num
    cells = []
    for i in range(start_timestep + 1, num_timesteps + 1, 1):
        try:
            cells.append(pd.read_csv(path + f"cells_av{constants['avg']}_timestep{i}.csv"))
        except FileNotFoundError as f:
            cells.append(pd.read_csv(path + f"cells_av{constants['avg']}_timestep0{i}.csv"))
    return cells
    

y = np.linspace(0, y_dim, 100)
plt.plot(y, parabolic_flow(y, y_dim), label = 'Parabolic')

data_dirs = [''] #400x50_0.01g_0.05t/', '400x50_0.01g_0.1t/', '400x50_0.01g_0.15t/', '400x50_0.01g_0.2t/'
for config in data_dirs:
    constants = getConstants(config)
    cells = getLastCells(config, 50)
    velocities_i = pd.DataFrame()
    for t, cells_t in enumerate(cells):
        cells_t.set_index(['i', 'j'], inplace = True)
        velocities_i[f'{t}'] = cells_t['meanX'].groupby('i').mean()
    plt.plot(velocities_i.mean(axis = 1), '-x', label = '$g = {}$, $\Delta t = {}$'.format(constants['g'], constants['delta_t']))
plt.legend(bbox_to_anchor=(1,1))
plt.title('x-velocity for different $\Delta t$')
plt.ylabel('$v_x$')
plt.xlabel('cell number')
#plt.savefig('Assets/poseuille.png')
plt.show()

In [ ]:
def getLastParticles(config):
    path = "Data/" + config
    constants = getConstants(config)
    particles = pd.read_csv(path + f"particles_av{constants['avg']}_timestep0{num_timesteps}.csv")
    return particles

def theoretical_dist(Delta_v, N_c, av_N_c = 10, m = 1, k_B = 1, T = 1):
    A = (m/(2 * math.pi * k_B * T * (1 - 1/N_c)))**(3/2)
    B = np.exp(-(m/(2 * k_B * T * (1 - 1/N_c))) * Delta_v**2)
    return A * B

def poisson_avg(Delta_v, max_sum = 1000, av_N_c = 10, m = 1, k_B = 1, T = 1):
    e = math.exp(-av_N_c)
    total = 0
    for N_c in range(2, max_sum):
        P = (av_N_c**N_c)/(math.factorial(N_c))
        N = theoretical_dist(Delta_v, N_c, m, k_B, T)
        total += P * (N)
    D = 1 - (av_N_c + 1) * e
    return e * total/D

data_dirs = [''] #400x50_0.01g_0.05t/, '400x50_0.01g_0.1t/', '400x50_0.01g_0.15t/', '400x50_0.01g_0.2t/'
fig, ax = plt.subplots(nrows = 1, ncols = 1)
for config in data_dirs:
    constants = getConstants(config)
    particles = getLastParticles(config)
    particles["i"] = (particles["y"] / constants["a"]).apply(lambda i: floor(i))
    particles["j"] = (particles["x"] / constants["a"]).apply(lambda i: floor(i))
    cells = getLastCell(config)
    complete = pd.merge(particles, cells, how = 'inner', on = ["i", "j"])
    complete["Delta_vx"] = complete["vx"] - complete["meanX"]
    complete["Delta_vy"] = complete["vy"] - complete["meanY"]
    complete["Delta_v"] = np.sqrt(complete["Delta_vx"]**2 + complete["Delta_vy"]**2)
    mid_i = (particles["i"] >= 1/4*y_dim) & (particles["i"] <= 35)
    subselection = complete.loc[mid_i]
    sns.kdeplot(subselection["Delta_v"]/ math.sqrt(constants["k_BT"] / constants["m"]),
                ax = ax, label = '$g = {}$, $\Delta t = {}$'.format(constants["g"], constants["delta_t"]))
    #ax.plot(subselection["Delta_v"], poisson_avg(subselection["Delta_v"]/ math.sqrt(constants["k_BT"] / constants["m"])), 'o')# ,ax = ax
ax.legend()
ax.set_title('Distribution of velocity in a cell')
ax.set_ylabel('Frequency')
ax.set_xlabel('$|\Delta v|$')
plt.savefig("Assets/Delta_v_dist.png")
plt.show()